## Getting started with BombCell

## Install bombcell

Create a conda environment
```bash
conda create -n bombcell python=3.11
conda activate bombcell
```
Clone latest bombcell repository from github
```bash
git clone https://github.com/Julie-Fabre/bombcell.git
```
Install bombcell from local repository
```bash
cd bombcell/pyBombCell
# you could do `pip install .`, but uv is much quicker!
pip install uv
uv pip install . # or uv pip install -e . (-e for editable mode)
```

## Imports

In [1]:
import os, sys
from pathlib import Path
from pprint import pprint 

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 50)
import matplotlib.pyplot as plt

In [2]:
# Optional

# # Add bombcell to Python path if NOT installed with pip
# # If notebook is running in bombcell repo:
# demo_dir = Path(os.getcwd())
# pyBombCell_dir = demo_dir.parent
# # Else:
# # pyBombCell_dir = "path/to/bombcell/repository/root"
# sys.path.append(str(pyBombCell_dir))

In [3]:
%load_ext autoreload
%autoreload 2

import bombcell as bc

## Define data paths

By default: path to BombCell's toy dataset

In [4]:
# Replace with your kilosort directory
ks_dir = "/home/julie/Dropbox/Example datatsets/JF093_2023-03-09_site1"

# Set bombcell's output directory
save_path = Path(ks_dir) / "bombcell"

In [5]:
## For Neuropixels probes, provide raw and meta files
# Leave 'None' if no raw data
raw_file_path = "/home/julie/Dropbox/Example datatsets/JF093_2023-03-09_site1/site1/2023-03-09_JF093_g0_t0_bc_decompressed.imec0.ap.bin" # ks_dir
meta_file_path = "/home/julie/Dropbox/Example datatsets/JF093_2023-03-09_site1/site1/2023-03-09_JF093_g0_t0.imec0.ap.meta"

In [6]:
## For non-Neuropixels probes, specify conversion factor to uV
# raw_file_path = "" # .bin, .dat...
# gain_to_uV = None # needs to be passed explicitly to bc.get_default_parameters
# sampling_rate = 30_000
# nChannels = 385
# n_bytes_per_sample = 2 # 2 bytes if int16, 4 if int32...

## Get parameters

In [7]:
param = bc.get_default_parameters(ks_dir, 
                                  raw_file=raw_file_path,
                                  meta_file=meta_file_path)

print("BombCell parameters:")
pprint(param)

Using previously decompressed ephys data file 2023-03-09_JF093_g0_t0_bc_decompressed.imec0.ap.bin as raw data.
BombCell parameters:
{'computeDistanceMetrics': False,
 'computeDrift': False,
 'computeSpatialDecay': True,
 'computeTimeChunks': False,
 'decompress_data': False,
 'deltaTimeChunk': 360,
 'detrendWaveform': True,
 'driftBinSize': 60,
 'duplicateSpikeWindow_s': 1e-05,
 'ephysKilosortPath': '/home/julie/Dropbox/Example '
                      'datatsets/JF093_2023-03-09_site1',
 'ephys_meta_file': '/home/julie/Dropbox/Example '
                    'datatsets/JF093_2023-03-09_site1/site1/2023-03-09_JF093_g0_t0.imec0.ap.meta',
 'ephys_sample_rate': 30000,
 'extractRaw': True,
 'gain_to_uV': 2.34375,
 'hillOrLlobetMethod': True,
 'isoDmin ': 20,
 'keep_only_somatic': True,
 'lratioMax': 0.1,
 'maxDrift': 100,
 'maxMainPeakToTroughRatio_nonSomatic': 0.8,
 'maxNPeaks': 2,
 'maxNTroughs': 1,
 'maxPeak1ToPeak2Ratio_nonSomatic': 3,
 'maxPercSpikesMissing': 20,
 'maxRPVviolations': 0.1

### Optionally customize parameters

In [8]:
# param["maxRPVviolations"] = 0.1
param["computeDistanceMetrics"] = 1
param["computeDrift"] = 1
# param["computeTimeChunks"] = 0
# ...
param['reextractRaw'] = False

## Run bombcell, get unit types and save results 

In [ ]:
(
    quality_metrics,
    param,
    unit_type,
    unit_type_string,
) = bc.run_bombcell(
    ks_dir, save_path, param
)

Loading file /home/julie/Dropbox/Example datatsets/JF093_2023-03-09_site1/bombcell/templates._bc_rawWaveforms.npy... Done!
No splits/merges detected


Computing bombcell quality metrics:   0%|          | 0/143 units

Reload quality metrics table

In [41]:
# quality metric values
quality_metrics_table = pd.DataFrame(quality_metrics)
quality_metrics_table.insert(0, 'Bombcell_unit_type', unit_type_string)
quality_metrics_table

,Bombcell_unit_type,phy_clusterID,nSpikes,nPeaks,nTroughs,waveformDuration_peakTrough,spatialDecaySlope,waveformBaselineFlatness,scndPeakToTroughRatio,mainPeakToTroughRatio,peak1ToPeak2Ratio,troughToPeak2Ratio,mainPeak_before_width,mainTrough_width,percentageSpikesMissing_gaussian,percentageSpikesMissing_symmetric,RPV_window_index,fractionRPVs_estimatedTauR,presenceRatio,maxDriftEstimate,cumDriftEstimate,rawAmplitude,signalToNoiseRatio,isolationDistance,Lratio,silhouetteScore,useTheseTimesStart,useTheseTimesStop,peak_channels
0,MUA,0,12531.0,1.0,1.0,300.000000,-0.025852,0.027249,0.397105,0.397105,0.068618,36.699207,NaN,3.810329,9.849033,39.281907,0.0,0.167895,1.000000,36.591587,140.981445,NaN,11.708648,18.243968,0.424067,NaN,0.003833,4387.514967,1
1,NOISE,1,27148.0,2.0,1.0,433.333333,-0.001318,0.282522,0.691792,0.691792,0.408391,3.539549,8.367749,10.218586,11.282480,32.063762,0.0,1.000000,1.000000,13.275711,154.386139,NaN,26.750210,10.602883,2.096542,NaN,0.003833,4387.514967,0
2,MUA,2,16230.0,2.0,1.0,600.000000,-0.022720,0.159244,0.345212,0.345212,0.461294,6.279655,11.822669,8.733908,100.000000,13.894636,0.0,1.000000,1.000000,88.453358,223.917088,NaN,7.521684,10.429920,2.111867,NaN,0.003833,4387.514967,1
3,MUA,3,5801.0,1.0,1.0,400.000000,-0.011218,0.066674,0.395594,0.395594,0.168540,14.998439,NaN,6.160191,10.814629,43.712401,0.0,0.956110,0.780822,29.536861,337.050392,NaN,10.851435,11.271841,2.140413,NaN,0.003833,4387.514967,2
4,MUA,4,26562.0,1.0,1.0,300.000000,-0.026757,0.029057,0.543130,0.543130,0.053499,34.415482,NaN,4.787313,1.443113,4.191314,0.0,0.085937,1.000000,30.199593,121.033363,NaN,13.760417,18.462630,0.413392,NaN,0.003833,4387.514967,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,MUA,138,5489.0,1.0,1.0,233.333333,-0.031424,0.020925,0.318595,0.318595,0.126107,24.889740,NaN,2.918676,9.885105,41.268992,0.0,0.189014,1.000000,28.169922,165.417969,NaN,31.176758,4.697311,109.926822,NaN,0.003833,4387.514967,125
139,MUA,139,5700.0,1.0,1.0,466.666667,-0.026308,0.073812,0.440791,0.440791,0.167454,13.547900,NaN,6.677262,10.336293,40.031562,0.0,0.308229,1.000000,32.089600,217.536865,NaN,7.526940,6.293041,35.527188,NaN,0.003833,4387.514967,138
140,NON-SOMA,140,99.0,2.0,1.0,400.000000,-0.066786,0.034273,0.255365,0.908673,3.558328,1.100506,2.388792,7.292709,100.000000,0.000000,0.0,0.000000,0.356164,74.517578,514.809448,NaN,45.131717,6.763300,17.134950,NaN,0.003833,4387.514967,149
141,MUA,141,151.0,1.0,1.0,466.666667,-0.033076,0.066439,0.410169,0.410169,0.171386,14.225288,15.599846,6.459551,0.361285,0.000000,0.0,0.000000,0.452055,59.163055,349.275726,NaN,10.874002,5.895531,416.241952,NaN,0.003833,4387.514967,30


In [11]:
# boolean table, if quality metrics pass threshold given parameters
boolean_quality_metrics_table = bc.make_qm_table(
    quality_metrics, param, unit_type_string
)
boolean_quality_metrics_table

,unit_type,Original ID,NaN result,# peaks,# troughs,duration,baseline flatness,peak2 / trough,spatial decay,# spikes,% spikes missing,presence ratio,fraction RPVs,amplitude,SNR,drift,non somatic,peak(main) / trough,peak1 / peak2,iso dist,l ratio
0,MUA,0,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,True
1,NOISE,1,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,True,True
2,MUA,2,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,True,True
3,MUA,3,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,True
4,MUA,4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,MUA,138,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,True
139,MUA,139,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,True
140,NON-SOMA,140,False,False,False,False,False,False,False,True,True,True,False,False,False,False,True,True,True,True,True
141,MUA,141,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,True,True


Example: get all quality metrics for unit 10

In [11]:
u = 12

units = quality_metrics_table.cluster_id
quality_metrics_table.loc[units == u, :]

,Bombcell_unit_type,cluster_id,n_spikes,n_peaks,n_troughs,waveform_duration_peak_trough,spatial_decay_slope,waveform_baseline_flatness,scnd_peak_to_trough_ratio,main_peak_to_trough_ratio,peak1_to_peak2_ratio,trough_to_peak2_ratio,peak_before_width,trough_width,percent_missing_gaussian,percent_missing_symmetric,RPV_use_tauR_est,fraction_RPVs,presence_ratio,max_drift_estimate,cumulative_drift_estimate,raw_amplitude,signal_to_noise_ratio,isolation_dist,l_ratio,silhouette_score,use_these_times_start,use_these_times_stop,peak_channels
12,GOOD,12,354.0,1.0,1.0,300.0,-0.013697,0.077735,0.49462,0.49462,0.088362,22.880278,NaN,6.52303,9.991433,34.926471,0.0,0.0,1.0,4.249339,6.090451,NaN,NaN,NaN,NaN,NaN,2880.325367,3240.325367,12
